In [ ]:
import requests

url = "https://live.trading212.com/api/v0/equity/portfolio"

headers = {"Authorization": ""}

response = requests.get(url, headers=headers)

data = response.json()
print(data)

In [ ]:
import pandas as pd

# Replace the file path with the location of your file
file_path = 'C:\\workarea\\files\\from_2024-01-25_to_2024-10-11_MTcyODY3NDAyMjMyNw.xlsx'

# Load the Excel file into a pandas DataFrame
df_excel = pd.read_excel(file_path)

# Display the first few rows to inspect the data
print(df_excel.head())


In [1]:
import pyodbc
import pandas as pd
from datetime import datetime
import settings  # Import settings from settings.py



# Function to connect to SQL Server
def connect_to_sql_server():
    conn_str = (
        #f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"DRIVER={{SQL Server}};"  # Use the existing 'SQL Server' driver
        f"SERVER={settings.server};"
        f"DATABASE={settings.database};"
        f"UID={settings.username};"
        f"PWD={settings.password}"
    )
    return pyodbc.connect(conn_str)

# Function to insert a new batch and return BatchID
def insert_batch(conn, file_name):
    cursor = conn.cursor()
    cursor.execute("INSERT INTO Batch (FileName, ProcessedTimestamp) OUTPUT INSERTED.BatchID VALUES (?, GETDATE())", (file_name,))
    batch_id = cursor.fetchone()[0]
    print ("Filename = ", file_name, "ProcessedTimestamp = ", "Batch ID = ", batch_id)
    conn.commit()
    return batch_id

In [2]:

# Function to insert deposit and interest transactions
def insert_deposit_interest(conn, df, batch_id):
    cursor = conn.cursor()
    deposits_df = df[df['Action'].isin(['Deposit', 'Interest on cash'])]  # Filter for deposits and interest
    for _, row in deposits_df.iterrows():
        cursor.execute("""
            INSERT INTO DepositInterest (BatchID, Action, Time, TotalAmount, CurrencyTotal, Notes) 
            VALUES (?, ?, ?, ?, ?, ?)
        """, (batch_id, row['Action'], row['Time'], row['Total'], row['Currency (Total)'], row['Notes']))
    print ("Inserted to Deposit Interest Table")
    conn.commit()

# Function to insert buy and sell transactions
def insert_buy_sell_transactions(conn, df, batch_id):
    cursor = conn.cursor()
    transactions_df = df[df['Action'].isin(['Market buy', 'Market sell'])]  # Filter for buy and sell transactions
    for _, row in transactions_df.iterrows():
        cursor.execute("""
            INSERT INTO BuySellTransactions (
                BatchID, Action, Time, ISIN, Ticker, Name, NoOfShares, PricePerShare, TotalAmount, CurrencyTotal, Notes
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (batch_id, row['Action'], row['Time'], row['ISIN'], row['Ticker'], row['Name'], row['No. of shares'], 
              row['Price / share'], row['Total'], row['Currency (Total)'], row['Notes']))
    print ("Inserted to buy_sell_transactions Table")
    conn.commit()

# Function to calculate and insert portfolio summary
def insert_portfolio_summary(conn, df, batch_id):
    cursor = conn.cursor()
    buy_transactions = df[df['Action'] == 'Market buy']
    portfolio = buy_transactions.groupby(['Ticker', 'Name']).agg(
        TotalShares=('No. of shares', 'sum'),
        AvgBuyPrice=('Price / share', 'mean')
    ).reset_index()

    for _, row in portfolio.iterrows():
        current_market_price = 100.0  # Placeholder for market price, can be updated dynamically
        profit_loss = (current_market_price - row['AvgBuyPrice']) * row['TotalShares']
        profit_loss_percentage = (profit_loss / (row['AvgBuyPrice'] * row['TotalShares'])) * 100
        
        cursor.execute("""
            INSERT INTO PortfolioSummary (
                BatchID, Ticker, Name, TotalShares, AvgBuyPrice, CurrentMarketPrice, ProfitLoss, ProfitLossPercentage
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (batch_id, row['Ticker'], row['Name'], row['TotalShares'], row['AvgBuyPrice'], 
              current_market_price, profit_loss, profit_loss_percentage))
    print ("Inserted to Portfolio Table")
    conn.commit()
# Load the CSV file
def load_csv(file_path):
    return pd.read_csv(file_path)

In [ ]:

# Main processing function
def process_file(file_path):
    # Connect to SQL Server
    conn = connect_to_sql_server()
    
    # Load CSV data
    global df
    df = load_csv(file_path)
    
    # Review data in chunks of 5 rows
    # review_data(df)

    # Insert Batch and get BatchID
    file_name = file_path.split('/')[-1]
    batch_id = insert_batch(conn, file_name)
    
    # Insert data into LandingData_Staging table
    #insert_staging_data(conn, df, batch_id)

    # # Insert data into tables
    # insert_landing_data(conn, df, batch_id)
    insert_deposit_interest(conn, df, batch_id)
    #insert_buy_sell_transactions(conn, df, batch_id)
    insert_portfolio_summary(conn, df, batch_id)
    
    # Close the connection
    conn.close()

# Example usage:
file_path = settings.filepath + 'from_2024-01-25_to_2024-10-11_MTcyODY3NDAyMjMyNw.csv'

# Run the process
process_file(file_path)

In [ ]:
conn.close()

In [ ]:
transactions_df = df[df['Action'].isin(['Market buy', 'Market sell'])]  # Filter for buy and sell transactions
transactions_df

In [ ]:
batch_id =0
for _, row in transactions_df.iterrows():
        print (batch_id, row['Action'], row['Time'], row['ISIN'], row['Ticker'], row['Name'], row['No. of shares'], 
              row['Price / share'], row['Total'], row['Currency (Total)'], row['Notes'])
        break
print ("Inserted to buy_sell_transactions Table")

In [ ]:
batch_id = 0
for _, row in transactions_df.iterrows():
    print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
    print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
    print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
    print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
    print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
    print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
    print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
    print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
    print(f"Total: {row['Total']} (Type: {type(row['Total']).__name__})")
    print(f"Currency (Total): {row['Currency (Total)']} (Type: {type(row['Currency (Total)']).__name__})")
    print(f"Notes: {row['Notes']} (Type: {type(row['Notes']).__name__})")
    break  # Remove this once you want to iterate through all rows

print("Inserted to buy_sell_transactions Table")


In [13]:
# Convert all values in the DataFrame to string
transactions_df = transactions_df.astype(str)


In [ ]:
batch_id = 0
for _, row in transactions_df.iterrows():
    print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
    print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
    print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
    print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
    print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
    print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
    print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
    print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
    print(f"Total: {row['Total']} (Type: {type(row['Total']).__name__})")
    print(f"Currency (Total): {row['Currency (Total)']} (Type: {type(row['Currency (Total)']).__name__})")
    print(f"Notes: {row['Notes']} (Type: {type(row['Notes']).__name__})")
    break  # Remove this once you want to iterate through all rows

print("Inserted to buy_sell_transactions Table")


In [ ]:
 # Insert each row into the LandingData table
conn = connect_to_sql_server()
cursor = conn.cursor()
for index, row in transactions_df.iterrows():
    try:
        # Assuming batch_id is defined and row is from iterating over transactions_df

        print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
        print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
        print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
        print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
        print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
        print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
        print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
        print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
        print(f"Currency (Price / share): {row['Currency (Price / share)']} (Type: {type(row['Currency (Price / share)']).__name__})")
        print(f"Exchange rate: {row['Exchange rate']} (Type: {type(row['Exchange rate']).__name__})")
        print(f"Total: {row['Total']} (Type: {type(row['Total']).__name__})")
        print(f"Currency (Total): {row['Currency (Total)']} (Type: {type(row['Currency (Total)']).__name__})")
        print(f"Withholding tax: {row['Withholding tax']} (Type: {type(row['Withholding tax']).__name__})")
        print(f"Currency (Withholding tax): {row['Currency (Withholding tax)']} (Type: {type(row['Currency (Withholding tax)']).__name__})")
        print(f"Stamp duty reserve tax: {row['Stamp duty reserve tax']} (Type: {type(row['Stamp duty reserve tax']).__name__})")
        print(f"Currency (Stamp duty reserve tax): {row['Currency (Stamp duty reserve tax)']} (Type: {type(row['Currency (Stamp duty reserve tax)']).__name__})")
        print(f"Notes: {row['Notes']} (Type: {type(row['Notes']).__name__})")
        print(f"ID: {row['ID']} (Type: {type(row['ID']).__name__})")
        print(f"Currency conversion fee: {row['Currency conversion fee']} (Type: {type(row['Currency conversion fee']).__name__})")
        print(f"Currency (Currency conversion fee): {row['Currency (Currency conversion fee)']} (Type: {type(row['Currency (Currency conversion fee)']).__name__})")
        #break
        cursor.execute("""
            INSERT INTO LandingData (
                BatchID, Action, Time, ISIN, Ticker, Name, NoOfShares, PricePerShare, 
                CurrencyPriceShare, ExchangeRate, TotalAmount, CurrencyTotal, 
                WithholdingTax, CurrencyWithholdingTax, StampDutyReserveTax, 
                CurrencyStampDuty, Notes, TransactionID, CurrencyConversionFee, 
                CurrencyConversionFeeCurrency
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            batch_id, row['Action'], row['Time'], row['ISIN'], row['Ticker'], row['Name'], 
            row['No. of shares'], row['Price / share'], row['Currency (Price / share)'], row['Exchange rate'],
            row['Total'], row['Currency (Total)'], row['Withholding tax'], row['Currency (Withholding tax)'], 
            row['Stamp duty reserve tax'], row['Currency (Stamp duty reserve tax)'], row['Notes'], 
            row['ID'], row['Currency conversion fee'], row['Currency (Currency conversion fee)']
        ))
        break
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)
conn.commit()
conn.close()

In [ ]:
conn = connect_to_sql_server()
cursor = conn.cursor()
for index, row in transactions_df.iterrows():
    try:
        # Print statement for debugging
        print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
        print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")

        # First insert with just BatchID and Action
        cursor.execute("""
            INSERT INTO LandingData_Staging (BatchID, Action)
            VALUES (?, ?)
        """, (batch_id, row['Action']))
        break
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)
conn.commit()
conn.close()


In [ ]:
conn = connect_to_sql_server()
cursor = conn.cursor()
for index, row in transactions_df.iterrows():
    try:
        # Print statement for debugging
        print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
        print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
        print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
        print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
        print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
        print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
        print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
        print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
        print(f"Currency (Price / share): {row['Currency (Price / share)']} (Type: {type(row['Currency (Price / share)']).__name__})")
        print(f"Exchange rate: {row['Exchange rate']} (Type: {type(row['Exchange rate']).__name__})")

        # Insert with more columns
        cursor.execute("""
            INSERT INTO LandingData_Staging (BatchID, Action, Time, ISIN, Ticker, Name, NoOfShares, PricePerShare, CurrencyPriceShare, ExchangeRate)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (batch_id, row['Action'], row['Time'], row['ISIN'], row['Ticker'], row['Name'], 
              row['No. of shares'], row['Price / share'], row['Currency (Price / share)'], row['Exchange rate']))
        break
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)
conn.commit()
conn.close()


In [ ]:
conn = connect_to_sql_server()
cursor = conn.cursor()
for index, row in transactions_df.iterrows():
    try:
        # Print statement for debugging each column
        print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
        print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
        print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
        print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
        print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
        print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
        print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
        print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
        print(f"Currency (Price / share): {row['Currency (Price / share)']} (Type: {type(row['Currency (Price / share)']).__name__})")
        print(f"Exchange rate: {row['Exchange rate']} (Type: {type(row['Exchange rate']).__name__})")
        print(f"Total: {row['Total']} (Type: {type(row['Total']).__name__})")
        print(f"Currency (Total): {row['Currency (Total)']} (Type: {type(row['Currency (Total)']).__name__})")
        print(f"Withholding tax: {row['Withholding tax']} (Type: {type(row['Withholding tax']).__name__})")
        print(f"Currency (Withholding tax): {row['Currency (Withholding tax)']} (Type: {type(row['Currency (Withholding tax)']).__name__})")
        print(f"Stamp duty reserve tax: {row['Stamp duty reserve tax']} (Type: {type(row['Stamp duty reserve tax']).__name__})")
        print(f"Currency (Stamp duty reserve tax): {row['Currency (Stamp duty reserve tax)']} (Type: {type(row['Currency (Stamp duty reserve tax)']).__name__})")
        print(f"Notes: {row['Notes']} (Type: {type(row['Notes']).__name__})")
        print(f"ID: {row['ID']} (Type: {type(row['ID']).__name__})")
        print(f"Currency conversion fee: {row['Currency conversion fee']} (Type: {type(row['Currency conversion fee']).__name__})")
        print(f"Currency (Currency conversion fee): {row['Currency (Currency conversion fee)']} (Type: {type(row['Currency (Currency conversion fee)']).__name__})")

        # Insert statement with all columns
        cursor.execute("""
            INSERT INTO LandingData_Staging (
                BatchID, Action, Time, ISIN, Ticker, Name, NoOfShares, PricePerShare, 
                CurrencyPriceShare, ExchangeRate, TotalAmount, CurrencyTotal, 
                WithholdingTax, CurrencyWithholdingTax, StampDutyReserveTax, 
                CurrencyStampDuty, Notes, TransactionID, CurrencyConversionFee, 
                CurrencyConversionFeeCurrency
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            batch_id, row['Action'], row['Time'], row['ISIN'], row['Ticker'], row['Name'], 
            row['No. of shares'], row['Price / share'], row['Currency (Price / share)'], row['Exchange rate'],
            row['Total'], row['Currency (Total)'], row['Withholding tax'], row['Currency (Withholding tax)'], 
            row['Stamp duty reserve tax'], row['Currency (Stamp duty reserve tax)'], row['Notes'], 
            row['ID'], row['Currency conversion fee'], row['Currency (Currency conversion fee)']
        ))

    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)
conn.commit()
conn.close()


In [34]:
conn = connect_to_sql_server()
cursor = conn.cursor()

# Counter to track rows inserted
row_count = 0

# Replace NaN with None (this will be inserted as NULL in SQL)
#transactions_df = df.where(pd.notnull(df), None)

# Convert all values in the DataFrame to string
transactions_df = transactions_df.astype(str)

# Truncate the table before loading new data
cursor.execute("TRUNCATE TABLE LandingData_Staging")
conn.commit()

for index, row in transactions_df.iterrows():
    try:
        # Print statement for debugging each column
        # print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
        # print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
        # print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
        # print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
        # print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
        # print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
        # print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
        # print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
        # print(f"Currency (Price / share): {row['Currency (Price / share)']} (Type: {type(row['Currency (Price / share)']).__name__})")
        # print(f"Exchange rate: {row['Exchange rate']} (Type: {type(row['Exchange rate']).__name__})")
        # print(f"Total: {row['Total']} (Type: {type(row['Total']).__name__})")
        # print(f"Currency (Total): {row['Currency (Total)']} (Type: {type(row['Currency (Total)']).__name__})")
        # print(f"Withholding tax: {row['Withholding tax']} (Type: {type(row['Withholding tax']).__name__})")
        # print(f"Currency (Withholding tax): {row['Currency (Withholding tax)']} (Type: {type(row['Currency (Withholding tax)']).__name__})")
        # print(f"Stamp duty reserve tax: {row['Stamp duty reserve tax']} (Type: {type(row['Stamp duty reserve tax']).__name__})")
        # print(f"Currency (Stamp duty reserve tax): {row['Currency (Stamp duty reserve tax)']} (Type: {type(row['Currency (Stamp duty reserve tax)']).__name__})")
        # print(f"Notes: {row['Notes']} (Type: {type(row['Notes']).__name__})")
        # print(f"ID: {row['ID']} (Type: {type(row['ID']).__name__})")
        # print(f"Currency conversion fee: {row['Currency conversion fee']} (Type: {type(row['Currency conversion fee']).__name__})")
        # print(f"Currency (Currency conversion fee): {row['Currency (Currency conversion fee)']} (Type: {type(row['Currency (Currency conversion fee)']).__name__})")

        # Insert statement with all columns
        cursor.execute("""
            INSERT INTO LandingData_Staging (
                BatchID, Action, Time, ISIN, Ticker, Name, NoOfShares, PricePerShare, 
                CurrencyPriceShare, ExchangeRate, TotalAmount, CurrencyTotal, 
                WithholdingTax, CurrencyWithholdingTax, StampDutyReserveTax, 
                CurrencyStampDuty, Notes, TransactionID, CurrencyConversionFee, 
                CurrencyConversionFeeCurrency
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            batch_id, row['Action'], row['Time'], row['ISIN'], row['Ticker'], row['Name'], 
            row['No. of shares'], row['Price / share'], row['Currency (Price / share)'], row['Exchange rate'],
            row['Total'], row['Currency (Total)'], row['Withholding tax'], row['Currency (Withholding tax)'], 
            row['Stamp duty reserve tax'], row['Currency (Stamp duty reserve tax)'], row['Notes'], 
            row['ID'], row['Currency conversion fee'], row['Currency (Currency conversion fee)']
        ))
        
        # Increment row counter
        row_count += 1

        # Commit after every 5 rows
        if row_count % 100 == 0:
            conn.commit()
            print(f"Committed after {row_count} rows")
            #break

    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")
        print(row)
         # Print statement for debugging each column
        print(f"BatchID: {batch_id} (Type: {type(batch_id).__name__})")
        print(f"Action: {row['Action']} (Type: {type(row['Action']).__name__})")
        print(f"Time: {row['Time']} (Type: {type(row['Time']).__name__})")
        print(f"ISIN: {row['ISIN']} (Type: {type(row['ISIN']).__name__})")
        print(f"Ticker: {row['Ticker']} (Type: {type(row['Ticker']).__name__})")
        print(f"Name: {row['Name']} (Type: {type(row['Name']).__name__})")
        print(f"No. of shares: {row['No. of shares']} (Type: {type(row['No. of shares']).__name__})")
        print(f"Price / share: {row['Price / share']} (Type: {type(row['Price / share']).__name__})")
        print(f"Currency (Price / share): {row['Currency (Price / share)']} (Type: {type(row['Currency (Price / share)']).__name__})")
        print(f"Exchange rate: {row['Exchange rate']} (Type: {type(row['Exchange rate']).__name__})")
        print(f"Total: {row['Total']} (Type: {type(row['Total']).__name__})")
        print(f"Currency (Total): {row['Currency (Total)']} (Type: {type(row['Currency (Total)']).__name__})")
        print(f"Withholding tax: {row['Withholding tax']} (Type: {type(row['Withholding tax']).__name__})")
        print(f"Currency (Withholding tax): {row['Currency (Withholding tax)']} (Type: {type(row['Currency (Withholding tax)']).__name__})")
        print(f"Stamp duty reserve tax: {row['Stamp duty reserve tax']} (Type: {type(row['Stamp duty reserve tax']).__name__})")
        print(f"Currency (Stamp duty reserve tax): {row['Currency (Stamp duty reserve tax)']} (Type: {type(row['Currency (Stamp duty reserve tax)']).__name__})")
        print(f"Notes: {row['Notes']} (Type: {type(row['Notes']).__name__})")
        print(f"ID: {row['ID']} (Type: {type(row['ID']).__name__})")
        print(f"Currency conversion fee: {row['Currency conversion fee']} (Type: {type(row['Currency conversion fee']).__name__})")
        print(f"Currency (Currency conversion fee): {row['Currency (Currency conversion fee)']} (Type: {type(row['Currency (Currency conversion fee)']).__name__})")


# Final commit for any remaining rows after the loop
if row_count % 5 != 0:
    conn.commit()
    print(f"Final commit after {row_count} rows")

conn.close()


Committed after 5 rows
Committed after 10 rows
Committed after 15 rows
Committed after 20 rows
Committed after 25 rows
Committed after 30 rows
Committed after 35 rows
Committed after 40 rows
Committed after 45 rows
Committed after 50 rows
Committed after 55 rows
Committed after 60 rows
Committed after 65 rows
Committed after 70 rows
Committed after 75 rows
Committed after 80 rows
Committed after 85 rows
Committed after 90 rows
Committed after 95 rows
Committed after 100 rows
Committed after 105 rows
Committed after 110 rows
Committed after 115 rows
Committed after 120 rows
Committed after 125 rows
Committed after 130 rows
Committed after 135 rows
Committed after 140 rows
Committed after 145 rows
Committed after 150 rows
Committed after 155 rows
Committed after 160 rows
Committed after 165 rows
Committed after 170 rows
Committed after 175 rows
Committed after 180 rows
Committed after 185 rows
Committed after 190 rows
Committed after 195 rows
Committed after 200 rows
Committed after 205 